# ```Part 01: INGESTION```

## Step 0: Setup the environment using ```Widgets```
--
- Input widgets allow you to add parameters to your notebooks and dashboards

In [0]:
# create the widget parameters 
dbutils.widgets.text("usernameWidget","","Enter Your Name Here: ")
dbutils.widgets.text("environmentWidget","","Enter Environment: ")

# save as variables 
usernameWidget = dbutils.widgets.get("usernameWidget")
environmentWidget = dbutils.widgets.get("environmentWidget")
print(usernameWidget)
print(environmentWidget)

Vinh
dev

## Step 1: Examine the raw data

In [0]:
# we have 4 years' worth of sales data to ingest 
display(dbutils.fs.ls("/mnt/00-mchan-demo/db-workshop-vn-2022/t0-raw-layer/"))

--------------------------------------------------------------------------- 
 ExecutionError Traceback (most recent call last)
 <command-2467854589205647> in <module> 
 1 # we have 4 years' worth of sales data to ingest 
 ----> 2 display ( dbutils . fs . ls ( "/mnt/00-mchan-demo/db-workshop-vn-2022/t0-raw-layer/" ) ) 

 /databricks/python_shell/dbruntime/dbutils.py in f_with_exception_handling (*args, **kwargs) 
 387 exc . __context__ = None 
 388 exc . __cause__ = None 
 --> 389 raise exc
 390 
 391 return f_with_exception_handling

 ExecutionError : An error occurred while calling z:com.databricks.backend.daemon.dbutils.FSUtils.ls.
: java.io.FileNotFoundException: No such file or directory: s3a://workspaces-us-east-1/nvirginia-prod/8745825485880134/mnt/00-mchan-demo/db-workshop-vn-2022/t0-raw-layer
	at shaded.databricks.org.apache.hadoop.fs.s3a.S3AFileSystem.s3GetFileStatus(S3AFileSystem.java:3361)
	at shaded.databricks.org.apache.hadoop.fs.s3a.S3AFileSystem.innerGetFileStatus(S3AFileSystem.java:3235)
	at shaded.databricks.org.apache.hadoop.fs.s3a.S3AFileSystem.getFileStatus(S3AFileSystem.java:3174)
	at shaded.databricks.org.apache.hadoop.fs.s3a.S3AFileSystem.innerListStatus(S3AFileSystem.java:2697)
	at shaded.databricks.org.apache.hadoop.fs.s3a.S3AFileSystem.lambda$listStatus$11(S3AFileSystem.java:2676)
	at shaded.databricks.org.apache.hadoop.fs.s3a.Invoker.once(Invoker.java:118)
	at shaded.databricks.org.apache.hadoop.fs.s3a.Invoker.once(Invoker.java:112)
	at shaded.databricks.org.apache.hadoop.fs.s3a.S3AFileSystem.listStatus(S3AFileSystem.java:2676)
	at com.databricks.backend.daemon.data.client.DBFSV2.$anonfun$listStatus$2(DatabricksFileSystemV2.scala:95)
	at com.databricks.s3a.S3AExceptionUtils$.convertAWSExceptionToJavaIOException(DatabricksStreamUtils.scala:70)
	at com.databricks.backend.daemon.data.client.DBFSV2.$anonfun$listStatus$1(DatabricksFileSystemV2.scala:92)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:330)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.scala:424)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperationWithResultTags$4(UsageLogging.scala:444)
	at com.databricks.logging.Log4jUsageLoggingShim$.$anonfun$withAttributionContext$1(Log4jUsageLoggingShim.scala:33)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(AttributionContext.scala:94)
	at com.databricks.logging.Log4jUsageLoggingShim$.withAttributionContext(Log4jUsageLoggingShim.scala:31)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:205)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:204)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.withAttributionContext(DatabricksFileSystemV2.scala:510)
	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:240)
	at com.databricks.logging.UsageLogging.withAttributionTags$(UsageLogging.scala:225)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.withAttributionTags(DatabricksFileSystemV2.scala:510)
	at com.databricks.logging.UsageLogging.recordOperationWithResultTags(UsageLogging.scala:419)
	at com.databricks.logging.UsageLogging.recordOperationWithResultTags$(UsageLogging.scala:339)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.recordOperationWithResultTags(DatabricksFileSystemV2.scala:510)
	at com.databricks.logging.UsageLogging.recordOperation(UsageLogging.scala:330)
	at com.databricks.logging.UsageLogging.recordOperation$(UsageLogging.scala:302)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.recordOperation(DatabricksFileSystemV2.scala:510)
	at com.databricks.backend.daemon.data.client.DBFSV2.listStatus(DatabricksFileSystemV2.scala:92)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystem.listStatus(DatabricksFileSystem.scala:164)
	at com.databricks.ba

## Step 2: Ingest the data using PySpark

In [0]:
# import relevant libraries 
from pyspark.sql.functions import *
from pyspark.sql.types import * 

# Path to the t0-raw-layer in S3
rawPath = "/mnt/00-mchan-demo/db-workshop-vn-2022/t0-raw-layer/"

# Ingest 4 years' worth of historical sales data from an e-commerce retailer 
df = (
  spark.read
       .format("csv")
       .option("header", "true")
       .option("sep", ",")
       .option("inferSchema", "true")
       .option("dateFormat", "yyyy-mm-dd")
       .load(rawPath)
)

In [0]:
# Examine the schema of the data 
df.printSchema()

In [0]:
display(df)

### Step 3: Add helper metadata columns

In [0]:
df = (
    df.withColumn("file_path", input_file_name())
      .withColumn("ingest_time", current_timestamp())
)

In [0]:
display(df.select("rowID","orderID", "file_path", "ingest_time").orderBy("rowID"))

### Step 4: Write the ingested data to the ```BRONZE``` layer table

In [0]:
spark.sql(f"CREATE DATABASE IF NOT EXISTS {environmentWidget}_{usernameWidget}_db")
spark.sql(f"USE {environmentWidget}_{usernameWidget}_db")

In [0]:
# basic pattern: df.write.mode("overwrite").saveAsTable("database.tableName")
(
  df.write
    .mode("overwrite")
    .partitionBy("state")
    .saveAsTable(f"{environmentWidget}_{usernameWidget}_db.t1_bronze_orders")
)

# -- END OF TASK 01 --